[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MaxMitre/Aplicaciones-Financieras/blob/main/Semana7/1_Manejo_de_portafolios_Markovitz.ipynb)

# Introducción

Trataremos de encontrar portafolios óptimos, utilizando como medidores la varianza de los retornos, así como el valor esperado de los retornos en un año (el valor que se espera ganar)

# Dependencias

In [ ]:
!pip install yfinance -U plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 93.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 13.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0
  Attempting uninstall:

In [ ]:
import yfinance as yf

import pandas as pd
import numpy as np

import cvxopt as opt

import plotly.express as px

# Datos


Los datos con los que trabajaremos son el precio de acciones de Google (GOOG), Apple (AAPL), IBM (IBM), Microsoft (MSFT) y ExxonMobil (XOM) del último año. 

Para obtener los datos, usaremos [```yfinance```](https://github.com/ranaroussi/yfinance) y nos centraremos en el precio de cierre ajustado (Adj Close).

In [ ]:
data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "PFE AAPL IBM MSFT XOM NFLX",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "1y",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "1d",

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        # group_by = 'ticker',
    ).loc[:, 'Adj Close']
data

[*********************100%***********************]  6 of 6 completed


,AAPL,IBM,MSFT,NFLX,PFE,XOM
Date,,,,,,
2022-01-24 00:00:00-05:00,160.687393,122.733109,293.618744,387.149994,49.918823,70.012596
2022-01-25 00:00:00-05:00,158.858017,129.669113,285.811890,366.420013,50.887367,72.070930
2022-01-26 00:00:00-05:00,158.768539,127.916046,293.955566,359.700012,51.342579,71.339943
2022-01-27 00:00:00-05:00,158.301254,126.258278,297.056519,386.700012,52.084274,72.253685
2022-01-28 00:00:00-05:00,169.347153,128.144699,305.398376,384.359985,53.021152,72.407578
...,...,...,...,...,...,...
2023-01-17 00:00:00-05:00,135.940002,145.190002,240.350006,326.220001,46.080002,112.930000
2023-01-18 00:00:00-05:00,135.210007,140.410004,235.809998,326.329987,45.000000,110.610001
2023-01-19 00:00:00-05:00,135.270004,140.619995,231.929993,315.779999,44.959999,111.320000


In [ ]:
data.shift()

,AAPL,IBM,MSFT,NFLX,PFE,XOM
Date,,,,,,
2022-01-24 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-25 00:00:00-05:00,160.687393,122.733109,293.618744,387.149994,49.918823,70.012596
2022-01-26 00:00:00-05:00,158.858017,129.669113,285.811890,366.420013,50.887367,72.070930
2022-01-27 00:00:00-05:00,158.768539,127.916046,293.955566,359.700012,51.342579,71.339943
2022-01-28 00:00:00-05:00,158.301254,126.258278,297.056519,386.700012,52.084274,72.253685
...,...,...,...,...,...,...
2023-01-17 00:00:00-05:00,134.759995,145.889999,239.229996,332.820007,47.849998,113.150002
2023-01-18 00:00:00-05:00,135.940002,145.190002,240.350006,326.220001,46.080002,112.930000
2023-01-19 00:00:00-05:00,135.210007,140.410004,235.809998,326.329987,45.000000,110.610001


Obtenemos los retornos logarítmicos anualizados para cada activo.

In [ ]:
annual_returns = np.log(data / data.shift()) / (1 / 252) # Para anualizar los retornos
annual_returns

,AAPL,IBM,MSFT,NFLX,PFE,XOM
Date,,,,,,
2022-01-24 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-25 00:00:00-05:00,-2.885398,13.853391,-6.790964,-13.868050,4.842572,7.301853
2022-01-26 00:00:00-05:00,-0.141980,-3.430164,7.079878,-4.664487,2.244237,-2.568989
2022-01-27 00:00:00-05:00,-0.742776,-3.287220,2.644437,18.239469,3.614350,3.207191
2022-01-28 00:00:00-05:00,16.997621,3.737274,6.979054,-1.529553,4.492624,0.536162
...,...,...,...,...,...,...
2023-01-17 00:00:00-05:00,2.197000,-1.212035,1.177043,-5.047515,-9.498392,-0.490449
2023-01-18 00:00:00-05:00,-1.356883,-8.436080,-4.805598,0.084948,-5.976575,-5.230929
2023-01-19 00:00:00-05:00,0.111797,0.376599,-4.180885,-8.281560,-0.224105,1.612404


In [ ]:
mean_returns = annual_returns.mean()
mean_returns

AAPL   -0.130960
IBM     0.145987
MSFT   -0.192478
NFLX   -0.080540
PFE    -0.105014
XOM     0.480399
dtype: float64

In [ ]:
cov_returns = annual_returns.cov()
cov_returns

,AAPL,IBM,MSFT,NFLX,PFE,XOM
AAPL,32.457680,10.687768,26.007997,31.310207,8.233499,9.086711
IBM,10.687768,14.348498,9.467135,0.589356,6.357780,6.625925
MSFT,26.007997,9.467135,31.897066,28.898362,8.762360,7.152982
NFLX,31.310207,0.589356,28.898362,129.331722,9.220578,10.340249
PFE,8.233499,6.357780,8.762360,9.220578,18.317374,5.646288
XOM,9.086711,6.625925,7.152982,10.340249,5.646288,30.771280


In [ ]:
mean_returns

AAPL   -0.130960
IBM     0.145987
MSFT   -0.192478
NFLX   -0.080540
PFE    -0.105014
XOM     0.480399
dtype: float64

# Markovitz

https://pyportfolioopt.readthedocs.io/en/latest/UserGuide.html

Si $w$ es el vector de pesos de las acciones con retornos esperados $\mu$, entonces el retorno del portafolio es igual al peso de cada acción multiplicado por su retorno, es decir, $w^T \mu$. El riesgo del portafolio en términos de la matriz de covarianzas $\Sigma$ esta dado por $\sigma^2 = w^T \Sigma w$. 

Lo cual, un poco mas "desmenuzado", es lo escrito en los 2 siguientes cuadros de texto.



## Valor esperado de portafolio comparado con valor esperado de cada activo 

Tengo 4 activos y un capital de $v_0$ pesos:
- cada activo tiene su retorno esperado $\mu_1, \, \mu_2, \, \mu_3, \, \mu_4 $
- a cada activo le invierto $w_1, \, w_2, \, w_3, \, w_4 $ ( son tales que $w_1 + w_2 + w_3 +  w_4 = v_0$)

Entonces:

$ \operatorname{E}(R_p) = \sum_i w_i \operatorname{E}(R_i) = \sum_i w_i \mu_i \quad $


## Volatilidad esperada de portafolio comparado con volatilidad de cada activo

- cada activo tiene una volatilida $\sigma_1, \, \sigma_2, \, \sigma_3, \, \sigma_4 $ respectivamente. Entonces la volatilidad del portafolio es:

$ \sigma_p^2 = \sum_i w_i^2 \sigma_{i}^2 + \sum_i \sum_{j \neq i} w_i w_j \sigma_i \sigma_j \rho_{ij} $,

donde $\sigma_i$ es la volatilidad de cada activo y $\rho_{ij}$ es el coeficiente de correlación de los retornos del activo $i$ con el activo $j$. 

La razón de Sharpe es el retorno en exceso del portafolio por unidad de riesgo (volatilidad)

$$
SR = \frac{R_p-R_f}{\sigma}
$$

Con esto en mente, crearemos un cantidad $N$ de portafolios con pesos aleatorios y guardaremos tanto los pesos, como el retorno, la volatilidad y la razon de Sharpe para cada uno.

**NOTA**: Los comentarios en los siguientes 2 bloques de código son para que al quitarlo veamos los portafolios PUROS (todo el capital a una sola acción)

Si dejan los comentarios -> obtienen las 10,000 muestras

Si quitan los comentarios -> ¿que obtienen? ________________

In [ ]:
np.random.seed(1995)

N = 10000
#N = 6  ##

k = annual_returns.shape[1]

weights = np.zeros((N, k))
returns = np.zeros(N)
volatilities = np.zeros(N)
sharpe_ratios = np.zeros(N)

weights2 = np.array([[1,0,0,0,0,0],[0,1,0,0,0,0],[0,0,1,0,0,0],[0,0,0,1,0,0],[0,0,0,0,1,0],[0,0,0,0,0,1]])

weights.shape

(10000, 6)

In [ ]:
weights

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [ ]:
for i in range(N):
    w = np.random.random(k)
    w /= np.sum(w)

    #w = weights2[i]  ##
    #weights[i, :] = weights2[i]  ##

    weights[i, :] = w

    returns[i] = np.dot(mean_returns, w)

    volatilities[i] = np.sqrt(np.dot(w.T, np.dot(cov_returns, w))) # w.T @ cov_returns @ w

    sharpe_ratios[i] = returns[i] / volatilities[i]

In [ ]:
weights

array([[0.0949638 , 0.10141735, 0.12112572, 0.15791162, 0.26267929,
        0.26190222],
       [0.09248761, 0.26057282, 0.15210978, 0.27501385, 0.06585835,
        0.15395759],
       [0.23176348, 0.25672011, 0.05838959, 0.21149406, 0.23724876,
        0.00438401],
       ...,
       [0.19475325, 0.21797501, 0.01656481, 0.30082045, 0.10398104,
        0.16590543],
       [0.35483007, 0.24082914, 0.1036472 , 0.03698297, 0.24320686,
        0.02050376],
       [0.18326347, 0.30669843, 0.10277785, 0.04338749, 0.02795858,
        0.33591417]])

In [ ]:
weights[1]

array([0.09248761, 0.26057282, 0.15210978, 0.27501385, 0.06585835,
       0.15395759])

In [ ]:
returns[1]

0.04154576131896085

In [ ]:
volatilities[1]

4.649347686205486

Graficamos la volatilidad contra el retorno de cada portafolio generado y coloreamos en función de la razón de Sharpe. Los portafolios (casi) óptimos, serían aquellos que tienen el mayor retorno para cierto nivel de volatilidad, o dicho de otra manera, los que tienen la menor volatilidad para algún retorno especificado.

In [ ]:
import matplotlib.pyplot as plt

px.scatter(x = volatilities, y = returns, color = sharpe_ratios,
           labels={
                     "x": "Volatilidad",
                     "y": "Retorno",
                     "color": "Razón de Sharpe"
                 }
           )
# plt.scatter(volatilities, returns, c = sharpe_ratios)

In [ ]:
import matplotlib.pyplot as plt

px.scatter(x = volatilities, y = returns, color = sharpe_ratios,
           labels={
                     "x": "Volatilidad",
                     "y": "Retorno",
                     "color": "Razón de Sharpe"
                 }
           )
# plt.scatter(volatilities, returns, c = sharpe_ratios)

> La optimización del portafolio se puede ver como un problema de optimización convexa y una solución puede encontrarse usando programación cuadrática. Si denotamos el retorno objetivo como $\mu^*$, el problema a resolver para el portafolio sólo con posiciones largas es:

\begin{align}
    \text{min}_w && w^T \Sigma w \\
    \text{s.a.} && w^t \mu \geq \mu^*  \\
    && w^T \mathbf{1} = 1 \\
    && w_i \geq 0
\end{align}

Para resolverlo, ocuparemos la función [```covxopt.solvers.qp```](https://cvxopt.org/userguide/coneprog.html#quadratic-programming). Esta requiere que el problema de optimización se encuentre en la forma general. A saber, la forma general de un problema de programación cuadrática es la siguiente:

\begin{align}
    \text{min}_x && \frac{1}{2}x^TPx + q^Tx \\
    \text{s.a.} && Gx \preceq h \\
    && Ax = b
\end{align}

In [ ]:
mu_star = .1

In [ ]:
G = opt.matrix(-np.concatenate([mean_returns.to_numpy().reshape(1, k),np.eye(k)]), tc = 'd')
h = opt.matrix(np.concatenate([np.array([-mu_star]).reshape((1, 1)), np.zeros((k, 1))]), tc = 'd')
q = opt.matrix(0.0, (k, 1))
A = opt.matrix(1.0, (1, k))
b = opt.matrix(1.0)
P = opt.matrix(2 * cov_returns.to_numpy(), tc = 'd')

In [ ]:
results = opt.solvers.qp(P, q, G, h, A, b)
results

     pcost       dcost       gap    pres   dres
 0:  1.0058e+01  8.9286e+00  1e+01  3e+00  4e+00
 1:  1.0071e+01  9.3684e+00  1e+00  1e-01  2e-01
 2:  1.0113e+01  1.0028e+01  9e-02  1e-16  6e-15
 3:  1.0104e+01  1.0100e+01  4e-03  2e-16  4e-15
 4:  1.0103e+01  1.0103e+01  1e-04  1e-16  7e-15
 5:  1.0103e+01  1.0103e+01  1e-06  8e-17  6e-15
Optimal solution found.


{'x': <6x1 matrix, tc='d'>,
 'y': <1x1 matrix, tc='d'>,
 's': <7x1 matrix, tc='d'>,
 'z': <7x1 matrix, tc='d'>,
 'status': 'optimal',
 'gap': 1.3644265262226372e-06,
 'relative gap': 1.3504558447978077e-07,
 'primal objective': 10.103452961741958,
 'dual objective': 10.103451597315432,
 'primal infeasibility': 7.888701498278355e-17,
 'dual infeasibility': 5.900110104926367e-15,
 'primal slack': 8.5635435394065e-08,
 'dual slack': 4.953866276286348e-08,
 'iterations': 5}

Los pesos del portafolio óptimo los obtenemos de la llave ```x```

In [ ]:
# impresion bonita de los numeros decimales
np.set_printoptions(suppress=True)

In [ ]:
w = np.asarray(results['x']).reshape((-1))
w

array([0.00000009, 0.4821165 , 0.02147188, 0.03707561, 0.30894625,
       0.15038967])

In [ ]:
data.columns

Index(['AAPL', 'IBM', 'MSFT', 'NFLX', 'PFE', 'XOM'], dtype='object')

In [ ]:
np.dot(mean_returns, w)

0.10306736917133455

La volatilidad de ```primal objective```

In [ ]:
np.sqrt(results['primal objective'])

3.1785929216780744

Si variamos los retornos objetivos, podemos obtener la **Frontera Eficiente**, que esta constituida de los portafolios óptimos para distintos niveles del retorno.

In [ ]:
mu_stars = np.linspace(.0, .29, 30)
mu_stars

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29])

In [ ]:
G = opt.matrix(-np.concatenate([mean_returns.to_numpy().reshape(1, k),np.eye(k)]), tc = 'd')
q = opt.matrix(0.0, (k, 1))
A = opt.matrix(1.0, (1, k))
b = opt.matrix(1.0)
P = opt.matrix(2 * cov_returns.to_numpy(), tc = 'd')

mu_stars = np.linspace(.0, .29, 30)

ws = np.zeros((len(mu_stars), k))
mus = np.zeros(len(mu_stars))
sigmas = np.zeros(len(mu_stars))

for i, mu_star in enumerate(mu_stars):
    try:
        h = opt.matrix(np.concatenate([np.array([-mu_star]).reshape((1, 1)), np.zeros((k, 1))]), tc = 'd')
        results = opt.solvers.qp(P, q, G, h, A, b)

        w = np.asarray(results['x']).reshape((-1))
        ws[i, :] = w
        mus[i] = np.dot(mean_returns, w)
        sigmas[i] = np.sqrt(results['primal objective'])
    except:
        print('domain error')

     pcost       dcost       gap    pres   dres
 0:  1.0059e+01  8.7924e+00  1e+01  3e+00  4e+00
 1:  1.0070e+01  9.2573e+00  1e+00  1e-01  2e-01
 2:  1.0111e+01  1.0014e+01  1e-01  9e-17  6e-15
 3:  1.0104e+01  1.0100e+01  4e-03  6e-17  4e-15
 4:  1.0103e+01  1.0103e+01  6e-05  8e-17  4e-15
 5:  1.0103e+01  1.0103e+01  6e-07  1e-16  6e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.0059e+01  8.8052e+00  1e+01  3e+00  4e+00
 1:  1.0070e+01  9.2661e+00  1e+00  1e-01  2e-01
 2:  1.0111e+01  1.0014e+01  1e-01  1e-16  7e-15
 3:  1.0104e+01  1.0100e+01  4e-03  2e-16  6e-15
 4:  1.0103e+01  1.0103e+01  6e-05  2e-16  1e-14
 5:  1.0103e+01  1.0103e+01  6e-07  6e-17  5e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.0059e+01  8.8181e+00  1e+01  3e+00  4e+00
 1:  1.0070e+01  9.2753e+00  1e+00  1e-01  2e-01
 2:  1.0111e+01  1.0013e+01  1e-01  8e-17  6e-15
 3:  1.0104e+01  1.0100e+01  4e-03  8e-17  5e-15
 4:  1.0103e+01  1.0103e

Finalmente, podemos graficar la frontera eficiente. Se puede observar que, pese a que las simulaciones nos dieron portafolios a la frontera, no eran realmente óptimos.

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_traces(
    [
        go.Scatter(
            x = volatilities, y = returns, 
            marker = dict(
                color = sharpe_ratios,
                colorbar = dict(title="Razón de Sharpe")
            ), 
            mode = 'markers', 
            showlegend = False
        ), 
        go.Scatter(
            x = sigmas, y = mus, 
            mode = 'lines + markers',  
            showlegend = False
        )
])

fig.update_layout(
    xaxis_title = 'Volatilidad',
    yaxis_title = 'Retorno'
)



Notas

1. Se están usando los retornos sin considerar la tasa libre de riesgo.

Ejercicios

1. Encontrar y graficar el portafolio óptimo de acuerdo a la razón de Sharpee 
2. Agregar los puntos de los portafolios que contienen únicamente un activo
3. Encontrar el portafolio con la menor volatilidad
4. Restar la tasa libre de riesgo a los retornos

# Mean-Variance Choice

El portafolio óptimo también se podría obtener maximizando, respecto a $w$,

$$
U(\mu, \Sigma; w) = w^T\mu - \frac{\delta}{2}w^T\Sigma w
$$

donde $\delta > 0$ es el parámetro de aversión al riesgo. La condición de primer orden para maximizarla es

$$
\mu = \delta \Sigma w
$$

lo que implica el siguiente diseño para un portafolio con riesgo:

$$
w = \left( \delta \Sigma \right)^{-1} \mu
$$

Es un sistema de ecuaciones lineales que podemos resolver con [```np.linalg.solve```](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html).

In [ ]:
delta = .1

np.linalg.solve(delta * cov_returns,  mean_returns)

array([-0.0807002 ,  0.18336906, -0.07860606,  0.02434859, -0.11210526,
        0.17112637])

In [ ]:
data.columns

Index(['AAPL', 'IBM', 'MSFT', 'NFLX', 'PFE', 'XOM'], dtype='object')

# Ligas interesantes

1. https://python-advanced.quantecon.org/black_litterman.html